<a href="https://colab.research.google.com/github/jsansao/teic-20231/blob/main/TEIC_Licao36bis_CoT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧠 Exemplo de Chain-of-Thought (CoT) com a API do Gemini

Este notebook demonstra como a técnica de **Engenharia de Prompt** chamada **Chain-of-Thought (CoT)** pode ser usada para melhorar drasticamente a capacidade do LLM de resolver problemas de lógica e aritmética.

**O que é CoT?** É uma técnica que força o modelo a "pensar passo a passo", gerando seu próprio raciocínio intermediário como texto, o que o ajuda a chegar na resposta correta.

**Vamos demonstrar:**
1.  Um prompt padrão (que provavelmente falhará).
2.  Um prompt CoT "Zero-Shot" (usando uma instrução simples).
3.  Um prompt CoT "Few-Shot" (dando um exemplo).

## Passo 1: Instalação e Configuração

Primeiro, instalamos a biblioteca e configuramos a API Key.

**Instruções (Importante):**
1.  Gere sua API Key no [Google AI Studio](https://aistudio.google.com/app/apikey).
2.  Clique no ícone de **chave** (🔑) na barra lateral esquerda do Colab.
3.  Adicione um novo segredo chamado `GOOGLE_API_KEY` e cole sua chave lá.

In [1]:
!pip install -q google-generativeai

In [2]:
import google.generativeai as genai
from google.colab import userdata
import os

try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=GOOGLE_API_KEY)
    print("API Key configurada com sucesso!")
except userdata.SecretNotFoundError:
    print("Erro: 'GOOGLE_API_KEY' não encontrada nos Secrets do Colab.")
    print("Por favor, adicione sua API Key na aba 'Secrets' (ícone de chave) à esquerda.")
except Exception as e:
    print(f"Ocorreu um erro: {e}")

API Key configurada com sucesso!


## Passo 2: O Problema

Vamos usar um problema clássico de lógica que requer múltiplos passos. LLMs que tentam responder "de uma vez" (inferência direta) geralmente erram.

In [3]:
# Definindo nosso modelo
model = genai.GenerativeModel('gemini-2.0-flash-lite')

# O problema
problema = "P: Se João tem 5 maçãs, dá 2 para Maria, que come 1 e Maria já tinha 3. Quantas maças tem Maria?  "

## Demonstração 1: Prompt Padrão (Sem CoT)

Vamos apenas fazer a pergunta. O modelo pode se confundir com os números (5, 2, 3, 1) e errar o cálculo final.

In [4]:
print("--- Tentativa 1: Prompt Padrão ---")
print(f"{problema}")

response_padrao = model.generate_content(problema)

print("\n--- Resposta do Gemini (Provavelmente Errada ou sem Raciocínio) ---")
try:
    print(response_padrao.text)
    print("\n(Nota: A resposta correta é 4. O modelo acertou ou errou?)")
except Exception as e:
    print(f"Erro: {e}")
    print(response_padrao.candidates)

--- Tentativa 1: Prompt Padrão ---
P: Se João tem 5 maçãs, dá 2 para Maria, que come 1 e Maria já tinha 3. Quantas maças tem Maria?  

--- Resposta do Gemini (Provavelmente Errada ou sem Raciocínio) ---
Maria tem 5 maçãs no final.

*   **Começo:** Maria tinha 3 maçãs.
*   **João dá:** João dá 2 maçãs para Maria, então ela passa a ter 3 + 2 = 5 maçãs.
*   **Maria come:** Maria come 1 maçã, mas isso não afeta o número total de maçãs que ela tem no momento em que estamos avaliando.
*   **Total:** Maria tem 5 maçãs.

(Nota: A resposta correta é 4. O modelo acertou ou errou?)


O Gemini está com "pensamento" ativado por default 😀

## Demonstração 2: Prompt CoT (Zero-Shot)

Agora, vamos adicionar a "frase mágica" que ativa o CoT. Para modelos em português, podemos usar algo como "Pense passo a passo."

In [5]:
prompt_cot_zero_shot = problema + "\n\nR: Pense passo a passo."

print("--- Tentativa 2: Prompt CoT (Zero-Shot) ---")
print(prompt_cot_zero_shot)

response_cot = model.generate_content(prompt_cot_zero_shot)

print("\n--- Resposta do Gemini (Com Raciocínio) ---")
try:
    print(response_cot.text)
except Exception as e:
    print(f"Erro: {e}")
    print(response_cot.candidates)

--- Tentativa 2: Prompt CoT (Zero-Shot) ---
P: Se João tem 5 maçãs, dá 2 para Maria, que come 1 e Maria já tinha 3. Quantas maças tem Maria?  

R: Pense passo a passo.

--- Resposta do Gemini (Com Raciocínio) ---
Vamos analisar o problema passo a passo:

1.  **Maria já tinha 3 maçãs:** Essa é a informação inicial sobre a quantidade de maçãs que Maria possuía.
2.  **João dá 2 maçãs para Maria:** Maria recebe mais 2 maçãs.
3.  **Maria come 1 maçã:** Maria come uma maçã, o que diminui a quantidade que ela tem.

Para calcular o total de maçãs que Maria tem, fazemos o seguinte:

*   Começamos com as 3 maçãs que ela já tinha.
*   Adicionamos as 2 que ela recebeu de João (3 + 2 = 5).
*   Subtraímos a maçã que ela comeu (5 - 1 = 4).

**Resposta:** Maria tem 4 maçãs.


## Demonstração 3: Prompt CoT (Few-Shot)

Uma técnica ainda mais robusta é dar ao modelo um exemplo (1-shot ou few-shot) de como pensar passo a passo. Isso é In-Context Learning (ICL) + CoT.

In [6]:
prompt_cot_few_shot = """P: Havia 4 carros no estacionamento. 3 mais chegaram e 1 saiu. Quantos carros há agora?
R: Havia 4 carros. 3 mais chegaram, então 4 + 3 = 7. 1 carro saiu, então 7 - 1 = 6. A resposta é 6.

P: Uma caixa tem 10 bolas. 5 são azuis e o resto é vermelho. Quantas são vermelhas?
R: A caixa tem 10 bolas no total. 5 são azuis. O resto é vermelho, então 10 - 5 = 5. A resposta é 5.

R:"""

print("--- Tentativa 3: Prompt CoT (Few-Shot) ---")
print(prompt_cot_few_shot)

response_cot_few_shot = model.generate_content(prompt_cot_few_shot)

print("\n--- Resposta do Gemini (Seguindo o Exemplo) ---")
try:
    print(response_cot_few_shot.text)
except Exception as e:
    print(f"Erro: {e}")
    print(response_cot_few_shot.candidates)

--- Tentativa 3: Prompt CoT (Few-Shot) ---
P: Havia 4 carros no estacionamento. 3 mais chegaram e 1 saiu. Quantos carros há agora?
R: Havia 4 carros. 3 mais chegaram, então 4 + 3 = 7. 1 carro saiu, então 7 - 1 = 6. A resposta é 6.

P: Uma caixa tem 10 bolas. 5 são azuis e o resto é vermelho. Quantas são vermelhas?
R: A caixa tem 10 bolas no total. 5 são azuis. O resto é vermelho, então 10 - 5 = 5. A resposta é 5.

R:

--- Resposta do Gemini (Seguindo o Exemplo) ---
P: João tinha 5 maçãs. Ele comeu 2. Quantas maçãs ele tem agora?
R: João tinha 5 maçãs. Ele comeu 2, então 5 - 2 = 3. A resposta é 3.



## Conclusão

Como você pode ver, as tentativas 2 e 3 não apenas têm maior probabilidade de acertar a resposta (4), mas também **mostram o raciocínio**.

**Por que funciona (Visão de DL):**
O modelo é um "next-token predictor".
1.  No **Prompt Padrão**, a "continuação mais provável" é apenas um número, que pode estar errado.
2.  No **Prompt CoT**, forçamos o modelo a gerar texto de raciocínio primeiro. Cada passo de raciocínio que ele gera (`Maria tinha 3...`) é adicionado ao contexto (via *self-attention*), tornando o *próximo* passo do raciocínio (`...ganhou 2, então 3+2=5...`) a continuação mais provável.

O CoT quebra um problema complexo em uma sequência de problemas fáceis.